In [0]:
from collections import Counter

import pickle
#import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from functools import partial
import scipy.cluster.hierarchy as sch
from sklearn.manifold import MDS
from sklearn.model_selection import train_test_split
from pyspark.sql import Row, SparkSession, Window


from pyspark.sql.types import StringType, IntegerType, LongType, FloatType, BooleanType, TimestampType, DataType


import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.types import *

import random

import sys


## Sandbox

In [0]:
dbutils.fs.mount(
  source = "wasbs://container-example@databricstoragexsgxlyrd.blob.core.windows.net",
  mount_point = "/mnt/blob-storage",
  extra_configs = {"fs.azure.account.key.databricstoragexsgxlyrd.blob.core.windows.net":
                    dbutils.secrets.get(scope = "databricks-secret-scope",
                                        key = "blob-container-key")})

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2335793403847658> in <module> 
 ----> 1 dbutils.fs.mount(
 2 source = "wasbs://container-example@databricstoragexsgxlyrd.blob.core.windows.net" , 
 3 mount_point = "/mnt/blob-storage" , 
 4 extra_configs = {"fs.azure.account.key.databricstoragexsgxlyrd.blob.core.windows.net":
 5 dbutils.secrets.get(scope = "databricks-secret-scope",

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o403.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/blob-storage; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/blob-storage
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/blob-storage
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:463)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:828)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:609)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:817)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:471)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:93)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:302)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:261)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:120)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:147)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:147)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:102)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(Us

In [0]:
# df1 = spark.read.parquet("/mnt/blob-storage/mappings_gameid.parquet")
# df2 = spark.read.parquet("/mnt/blob-storage/mappings_team.parquet")

In [0]:
# df2 = df2.withColumn('row_num', f.monotonically_increasing_id()).withColumn("row_num", f.col("row_num").cast(StringType())).withColumn('name', f.concat(f.lit("Team "),f.col('row_num'))).withColumn('abbrev', f.concat(f.lit("Team "),f.col('row_num'))).drop('row_num')
# display(df2)

id,nbaId,intId,name,abbrev,insertTime
feb4cb57-89ef-11e6-86b2-a45e60e298d3,1610612738,null,Team 0,Team 0,2022-05-20T04:05:17.286+0000
feb3e09c-89ef-11e6-9f68-a45e60e298d3,1610612737,null,Team 1,Team 1,2022-05-20T04:05:17.286+0000
feb4d7b8-89ef-11e6-80f2-a45e60e298d3,1610612757,null,Team 2,Team 2,2022-05-20T04:05:17.286+0000
feb4cdf5-89ef-11e6-9ab9-a45e60e298d3,1610612741,null,Team 3,Team 3,2022-05-20T04:05:17.286+0000
feb4ceb0-89ef-11e6-9f6d-a45e60e298d3,1610612739,null,Team 4,Team 4,2022-05-20T04:05:17.286+0000
feb4d402-89ef-11e6-b582-a45e60e298d3,1610612749,null,Team 5,Team 5,2022-05-20T04:05:17.286+0000
feb4d935-89ef-11e6-b12d-a45e60e298d3,1610612761,null,Team 6,Team 6,2022-05-20T04:05:17.286+0000
feb4d9a3-89ef-11e6-a904-a45e60e298d3,1610612762,null,Team 7,Team 7,2022-05-20T04:05:17.286+0000
feb4d217-89ef-11e6-a043-a45e60e298d3,1610612746,null,Team 8,Team 8,2022-05-20T04:05:17.286+0000
feb4d6d1-89ef-11e6-a925-a45e60e298d3,1610612755,null,Team 9,Team 9,2022-05-20T04:05:17.286+0000


In [0]:
# df1 = spark.read.parquet("/mnt/blob-storage/mappings_team.parquet")

In [0]:
# df2 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/mrzutshi@microsoft.com/mappings_gameid-2.csv")

In [0]:
# df1.write.mode('overwrite').parquet("/mnt/blob-storage/mappings_gameid.parquet")
# df2.write.parquet("/mnt/blob-storage/mappings_team.parquet", mode="overwrite")

In [0]:
df = spark.read.parquet("/mnt/blob-storage/tracking_data.parquet")

In [0]:
# df = df.withColumn("dividerId", f.col("dividerId").cast(IntegerType()))
# df = df.withColumn("teamId", f.col("teamId").cast(IntegerType()))
# df = df.withColumn("period", f.col("period").cast(IntegerType()))
# df = df.withColumn("x_grid", f.col("x_grid").cast(IntegerType()))
# df = df.withColumn("y_grid", f.col("y_grid").cast(IntegerType()))
# df = df.withColumn("ball_x_grid", f.col("ball_x_grid").cast(IntegerType()))
# df = df.withColumn("ball_y_grid", f.col("ball_y_grid").cast(IntegerType()))
# df = df.withColumn("fta", f.col("fta").cast(IntegerType()))

# df = df.withColumn("ball_y", f.col("ball_y").cast(FloatType()))
# df = df.withColumn("x", f.col("x").cast(FloatType()))
# df = df.withColumn("ball_x", f.col("ball_x").cast(FloatType()))
# df = df.withColumn("y", f.col("y").cast(FloatType()))
# df = df.withColumn("gcTime", f.col("gcTime").cast(FloatType()))
# df = df.withColumn("dx", f.col("dx").cast(FloatType()))
# df = df.withColumn("dy", f.col("dy").cast(FloatType()))
# df = df.withColumn("dy", f.col("dividerId").cast(FloatType()))
# df = df.withColumn("ballDx", f.col("ballDx").cast(FloatType()))
# df = df.withColumn("ballDy", f.col("ballDy").cast(FloatType()))
# df = df.withColumn("basketX", f.col("basketX").cast(FloatType()))

# df = df.withColumn("wcTime", f.col("wcTime").cast(LongType()))

# df = df.withColumn("hasPoss", f.col("hasPoss").cast(BooleanType()))

In [0]:
display(df)

gameId,wcTime,playerId,teamId,period,possWcStart,possWcEnd,possTeam,frameId,gcTime,scTime,x,y,ball_x,ball_y,ball_z,basketX,speed,area,cx,cy,teamCentroidDistBasket,distToBall,ballDistanceBasket,touch_wcStart,event_wcTrackStart,has_ball,divider_id,end_of_subsequence,reason_for_new_subsequence,reason_for_end_subsequence,subsequence_sent,created_by,date,version,possId,fta,outcome,hasPoss
0022000391,1613094446002,1628372,1610612765,2,1613094442842,1613094451082,1610612765,131127,380.24,22.84,18.33,-13.3,25.46,13.54,4.29,-41.75,12.0649,399.56934,17.52,3.5025,59.37,27.77,68.56,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,true
0022000391,1613094446002,1627763,1610612754,2,1613094442842,1613094451082,1610612765,131127,380.24,22.84,-1.83,-14.24,25.46,13.54,4.29,-41.75,9.5033,260.8474,4.575,0.9000002,46.33,38.94,68.56,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,false
0022000391,1613094447122,203087,1610612754,2,1613094442842,1613094451082,1610612765,131155,379.12,21.72,-18.24,9.12,-17.05,4.54,10.29,-41.75,18.9275,293.766,-9.74,-0.61000013,32.02,4.73,25.11,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,false
0022000391,1613094447122,203924,1610612765,2,1613094442842,1613094451082,1610612765,131155,379.12,21.72,2.25,-19.89,-17.05,4.54,10.29,-41.75,16.002,644.97455,-0.5799999,-1.134,41.19,31.13,25.11,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,true
0022000391,1613094447122,1628988,1610612754,2,1613094442842,1613094451082,1610612765,131155,379.12,21.72,-1.21,8.53,-17.05,4.54,10.29,-41.75,11.8242,293.766,-9.74,-0.61000013,32.02,16.33,25.11,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,false
0022000391,1613094447122,1630191,1610612765,2,1613094442842,1613094451082,1610612765,131155,379.12,21.72,-18.22,0.94,-17.05,4.54,10.29,-41.75,21.7557,644.97455,-0.5799999,-1.134,41.19,3.79,25.11,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,true
0022000391,1613094447122,1628367,1610612765,2,1613094442842,1613094451082,1610612765,131155,379.12,21.72,12.28,7.88,-17.05,4.54,10.29,-41.75,11.1636,644.97455,-0.5799999,-1.134,41.19,29.52,25.11,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,true
0022000391,1613094447122,1630180,1610612765,2,1613094442842,1613094451082,1610612765,131155,379.12,21.72,-3.88,20.41,-17.05,4.54,10.29,-41.75,18.8845,644.97455,-0.5799999,-1.134,41.19,20.62,25.11,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,true
0022000391,1613094447122,1627734,1610612754,2,1613094442842,1613094451082,1610612765,131155,379.12,21.72,-12.67,2.17,-17.05,4.54,10.29,-41.75,21.3219,293.766,-9.74,-0.61000013,32.02,4.98,25.11,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,false
0022000391,1613094447122,204456,1610612754,2,1613094442842,1613094451082,1610612765,131155,379.12,21.72,-2.7,-8.42,-17.05,4.54,10.29,-41.75,11.275,293.766,-9.74,-0.61000013,32.02,19.34,25.11,null,null,0,162,0,possession change,shot taken,0,Mrinal,2021-10-25,1.0,6424512b-18d9-56dd-89c8-89d4f51133ff,0,FGM,false


In [0]:
df.count()

Out[7]: 10861456

In [0]:

#Court Class
class Court:
  def __init__(self):
    
    self.schema_info = {"dividerId":
                          {"type": IntegerType(), "func": lambda x: x.iloc[0].dividerId},
                        "teamId":
                          {"type": IntegerType(), "func": lambda x: x.iloc[0].teamId},
                        "startWcTime": 
                          {"type": LongType(), "func": lambda x: min(x.start_wctime)},
                        "endWcTime": 
                          {"type": LongType(), "func": lambda x: max(x.start_wctime)},
                        "startGcTime": 
                          {"type": FloatType(), "func": lambda x: max(x.start_gctime)},
                        "endGcTime": 
                          {"type": FloatType(), "func": lambda x: min(x.start_gctime)},
                        "period": 
                          {"type": IntegerType(), "func": lambda x: x.iloc[0].period},
                        "gameId": 
                          {"type": StringType(), "func": lambda x: x.iloc[0].gameId},
                        "possId":
                          {"type": StringType(), "func": lambda x: x.iloc[0].possId},
#                         "wcDuration": 
#                           {"type": IntegerType(), "func": lambda x: x.iloc[0].wcDuration},
#                         "gcDuration": 
#                           {"type": IntegerType(), "func": lambda x: x.iloc[0].gcDuration},
                        "StartDivider": 
                          {"type": StringType(), "func": lambda x: x.iloc[0].StartDivider},
                        "EndDivider": 
                          {"type": StringType(), "func": lambda x: x.iloc[0].EndDivider},
                        "outcome": 
                          {"type": StringType(), "func": lambda x: x.iloc[0].outcome},
                        "hasPoss": 
                          {"type": BooleanType(), "func": lambda x: x.iloc[0].hasPoss},
                        "basketX":
                          {"type": FloatType(), "func": lambda x: x.iloc[0].basketX},
                        "fta":
                          {"type": IntegerType(), "func": lambda x: x.iloc[0].fta},
                        "court": 
                          {"type": ArrayType(FloatType()), "func": lambda x: x.values.ravel().tolist()}
                       }

  @property
  def output_func_dict(self):
    return {k: v["func"] for k,v in self.schema_info.items()}
  
  @property
  def output_schema(self):
    return StructType([StructField(k, v["type"]) for k,v in self.schema_info.items()])
    
  @staticmethod
  def get_court(data, output_func_dict, agg_col_name="count", divider=False, ball=False, v2 = False, expbh = False):

    if v2:
      df_court = data[["v_x_grid", "v_y_grid", agg_col_name]].pivot(
                  index='v_x_grid', columns='v_y_grid', values=agg_col_name).fillna(0.0)
    elif ball:
      df_court = data[["ball_x_grid", "ball_y_grid", agg_col_name]].pivot(
                  index='ball_x_grid', columns='ball_y_grid', values=agg_col_name).fillna(0.0)
    elif expbh:
      df_court = data[["x_grid_expbh", "y_grid_expbh", agg_col_name]].pivot(
                  index='x_grid_expbh', columns='y_grid_expbh', values=agg_col_name).fillna(0.0)  
    else:
      df_court = data[["x_grid", "y_grid", agg_col_name]].pivot(
                  index='x_grid', columns='y_grid', values=agg_col_name).fillna(0.0)

    # We can use the same set up here, as we have already scaled our velocities to fit withint the range that position fits in.
    missing_indexes = [x for x in list(range(-50,51)) if x not in df_court.index.tolist()]
    missing_columns = [x for x in list(range(-25,26)) if x not in df_court.columns.tolist()]

    for missing_index in missing_indexes:
      df_court.loc[missing_index,:] = 0.0

    for missing_column in missing_columns:
      df_court.loc[:,missing_column] = 0.0

    df_court = df_court.reindex(index=list(range(-50,51)))
    df_court = df_court.reindex(columns=list(range(-25,26)))
    
    print(df_court.count())
    #return df_court
    return pd.DataFrame([{**{k: v(data) for k,v in output_func_dict.items() if k != "court"},
                         **{k: v(df_court) for k,v in output_func_dict.items() if k == "court"}}])

In [0]:
def groupby_to_courts(just_xy, velocity=False, divider=False, ball=False, v2 = False, expbh = False):
  
  if v2:
    groupby_cols = ["gameId", "possId", "teamId", "hasPoss", "outcome", "dividerId", "v_x_grid", "v_y_grid", "fta", "basketX"]
  elif ball:
    groupby_cols = ["gameId", "possId", "teamId", "hasPoss", "outcome", "dividerId", "ball_x_grid", "ball_y_grid","fta", "basketX"]
  elif expbh:
    groupby_cols = ["gameId", "possId", "teamId", "hasPoss", "outcome", "dividerId", "x_grid_expbh", "y_grid_expbh" ,"fta", "basketX"]
  else:
    groupby_cols = ["gameId", "possId", "teamId", "hasPoss", "outcome", "dividerId", "x_grid", "y_grid", "fta", "basketX"]
  
  
  df_all_courts = ( just_xy.groupBy(groupby_cols)
                       .agg( f.min("wcTime").alias("start_wctime"),
                             f.max("wcTime").alias("end_wctime"),
                             f.max("gcTime").alias("start_gctime"),
                             f.min("gcTime").alias("end_gctime"),
                             f.first("period").alias("period"),
                             f.first("reason_for_new_subsequence").alias("StartDivider"),
                             f.first("reason_for_end_subsequence").alias("EndDivider"),
                             f.count("dx").alias("count"), #how many rows got grouped (dx is pretty meaningless here) any column thats not grouped by
                             f.sum(f.abs("dx")).alias("dx_sum"), 
                             f.sum(f.abs("dy")).alias("dy_sum"),
                             f.sum(f.abs("dx") + f.abs("dy")).alias("dxy_sum")) )
  
  
  return df_all_courts


In [0]:
df_all = {"courts": groupby_to_courts(df, velocity=False, divider=True),
         "v2": groupby_to_courts(df, velocity = False, divider = True, v2 = True),
          "balls": groupby_to_courts(df, velocity=False, divider=True, ball=True)}

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2335793403847667> in <module> 
 ----> 1 df_all = {"courts": groupby_to_courts(df, velocity=False, divider=True),
 2 "v2" : groupby_to_courts ( df , velocity = False , divider = True , v2 = True ) , 
 3 "balls": groupby_to_courts(df, velocity=False, divider=True, ball=True)}

 <command-2335793403847666> in groupby_to_courts (just_xy, velocity, divider, ball, v2, expbh) 
 11 
 12 
 ---> 13 df_all_courts = ( just_xy.groupBy(groupby_cols)
 14 .agg( f.min("wcTime").alias("start_wctime"),
 15 f . max ( "wcTime" ) . alias ( "end_wctime" ) , 

 /databricks/spark/python/pyspark/sql/group.py in agg (self, *exprs) 
 116 # Columns 
 117 assert all ( isinstance ( c , Column ) for c in exprs ) , "all exprs should be Column" 
 --> 118 jdf = self._jgd.agg(exprs[0]._jc,
 119 _to_seq(self.sql_ctx._sc, [c._jc for c in exprs[1:]]))
 120 return DataFrame ( jdf , self . sql_ctx ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Column 'dividerId' does not exist. Did you mean one of the following? [divider_id, playerId, frameId, gameId, date, gcTime, period, possId, scTime, speed, teamId, wcTime, area, basketX, distToBall, possTeam, possWcEnd, version, ball_x, ball_y, ball_z, created_by, cx, cy, fta, hasPoss, has_ball, outcome, x, y, possWcStart, touch_wcStart, subsequence_sent, ballDistanceBasket, end_of_subsequence, event_wcTrackStart, teamCentroidDistBasket, reason_for_end_subsequence, reason_for_new_subsequence];
'Aggregate [gameId#26186, possId#26221, teamId#26189, hasPoss#26224, outcome#26223, 'dividerId, 'x_grid, 'y_grid, fta#26222, basketX#26202], [gameId#26186, possId#26221, teamId#26189, hasPoss#26224, outcome#26223, 'dividerId, 'x_grid, 'y_grid, fta#26222, basketX#26202, min(wcTime#26187L) AS start_wctime#26383L, max(wcTime#26187L) AS end_wctime#26385L, max(gcTime#26195) AS start_gctime#26387, min(gcTime#26195) AS end_gctime#26389, first(period#26190, false) AS period#26391, first(reason_for_new_subsequence#26215, false) AS StartDivider#26393, first(reason_for_end_subsequence#26216, false) AS EndDivider#26395, count('dx) AS count#26397, sum(abs('dx, false)) AS dx_sum#26399, sum(abs('dy, false)) AS dy_sum#26401, sum((abs('dx, false) + abs('dy, false))) AS dxy_sum#26403]
+- Relation [gameId#26186,wcTime#26187L,playerId#26188,teamId#26189,period#26190,possWcStart#26191,possWcEnd#26192,possTeam#26193,frameId#26194,gcTime#26195,scTime#26196,x#26197,y#26198,ball_x#26199,ball_y#26200,ball_z#26201,basketX#26202,speed#26203,area#26204,cx#26205,cy#26206,teamCentroidDistBasket#26207,distToBall#26208,ballDistanceBasket#26209,... 15 more fields] parquet

In [0]:
additional_label = '0616'
if additional_label:
  additional_label = '_' + additional_label
else:
  additional_label = ''
  

#The below flags are really names for the experimental configuration, i.e. the "v2" experiment was decided to be [position, v2, ball], and the expbh experiement was [position, dx, dy, ball, expbh]
#Currently this is not configured to take in v2 AND expbh
v2 = False
expbh = False


court_obj = Court()


if v2:
  save_jig = \
           [
              {"col_name": "count", "table_name": "new_play_v2_positionmaps" + additional_label, "input": "courts", "divider": True, "ball": False, 'v2': False, 'expbh': False},
              {"col_name": "count", "table_name": "new_play_v2_velocitymaps" + additional_label, "input": "v2", "divider": True, "ball": False, 'v2': True, 'expbh': False},
              {"col_name": "count", "table_name": "new_play_v2_positionball" + additional_label, "input": "balls", "divider": True, "ball": True, 'v2': False, 'expbh': False}
           ]

elif expbh:
  save_jig = \
           [
              {"col_name": "count", "table_name": "new_play_positionmaps" + additional_label, "input": "courts", "divider": True, "ball": False, 'v2': False, 'expbh': False},
              {"col_name": "dx_sum", "table_name": "new_play_velocitymaps_dx" + additional_label, "input": "courts", "divider": True, "ball": False, 'v2': False, 'expbh': False},
              {"col_name": "dy_sum", "table_name": "new_play_velocitymaps_dy" + additional_label, "input": "courts", "divider": True, "ball": False, 'v2': False, 'expbh': False},
              {"col_name": "count", "table_name": "new_play_expbh" + additional_label, "input": "expbh", "divider": True, "ball": False, 'v2': False, 'expbh': True},
              {"col_name": "count", "table_name": "new_play_positionball" + additional_label, "input": "balls", "divider": True, "ball": True, 'v2': False, 'expbh': False}
           ]
  
else:
  save_jig = [
              {"col_name": "count", "table_name": "new_play_positionmaps" + additional_label, "input": "courts", "divider": True, "ball": False, 'v2': False, 'expbh': False},
              {"col_name": "dx_sum", "table_name": "new_play_velocitymaps_dx" + additional_label, "input": "courts", "divider": True, "ball": False, 'v2': False, 'expbh': False},
              {"col_name": "dy_sum", "table_name": "new_play_velocitymaps_dy" + additional_label, "input": "courts", "divider": True, "ball": False, 'v2': False, 'expbh': False},
              {"col_name": "dxy_sum", "table_name": "new_play_velocitymaps_dxy" + additional_label, "input": "courts", "divider": True, "ball": False, 'v2': False, 'expbh': False},
              {"col_name": "count", "table_name": "new_play_positionball" + additional_label, "input": "balls", "divider": True, "ball": True, 'v2': False, 'expbh': False}
           ]



for jig in save_jig:
  partial_get_court = partial(court_obj.get_court, output_func_dict=court_obj.output_func_dict,
                                                   agg_col_name=jig["col_name"], 
                                                   divider=jig["divider"], 
                                                   ball=jig["ball"],
                                                   v2=jig["v2"],
                                                   expbh=jig["expbh"])

  df_heatmaps = (df_all[jig["input"]].groupBy("teamId", "possId", "dividerId")
                                      .applyInPandas(partial_get_court, schema=court_obj.output_schema) )

  
  df_heatmaps.write.mode("overwrite").saveAsTable(f'default.{jig["table_name"]}')  #this writes out heatmaps, comment out if you want to play with these

In [0]:
display(df_heatmaps)

dividerId teamId startWcTime endWcTime startGcTime endGcTime period gameId possId StartDivider EndDivider outcome hasPoss basketX fta court 361 1610612737 1609208993454 1609209000694 472.28 465.04 4 0022000041 94e9aa39-1533-50fa-90e4-8244cac2319b time discontinuous shot taken FGM true 41.75 0 List(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0